In [2]:
import pandas as pd
import seaborn as sns
import seaborn.objects as so
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut

In [3]:
data_raw = pd.read_csv("FBRef2020-21.csv")

In [4]:
data_proc = (data_raw.query("Min >= 500")
             .drop(columns = [column for column in data_raw.columns if data_raw[column].isna().sum() > 100]))

display(data_proc[data_proc.isna().sum(axis=1) != 0])

display(data_proc.loc[313].isna().sum())
display(data_proc.loc[1190].isna().sum())
display(data_proc.loc[2525].isna().sum())
display(data_proc.loc[2525].isna().sort_values())


data_proc = data_proc.drop(index=[313,1190,2525]).reset_index()

,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,...,PrgDistCarry/90,ProgCarry/90,CarryIntoThird/90,CarryIntoBox/90,Miscontrol/90,Dispossessed/90,PassTarget/90,PassesReceived/90,PassRec%,ProgPassReceived/90
313,Salvador Ferrer,es ESP,DF,Spezia,it Serie A,22.0,1998.0,18,14,1190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1190,Jota,pt POR,FW,Valladolid,es La Liga,21.0,1999.0,17,7,668,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2525,Martín Zubimendi,es ESP,MF,Real Sociedad,es La Liga,21.0,1999.0,31,17,1882,...,131.4,4.74,1.44,0.05,0.33,0.72,43.9,42.5,96.8,0.86


95

95

2

Player                 False
Att 3rdPress/90        False
Mid 3rdPress/90        False
Def 3rdPress/90        False
SuccPress/90           False
                       ...  
MedAtt/90              False
FKPassAtt/90           False
ProgPassReceived/90    False
ErrToShot/90            True
Clearances/90           True
Name: 2525, Length: 119, dtype: bool

In [5]:
data_num = data_proc.drop(columns = [col for col in data_proc.columns if data_proc[col].dtype not in ['float64','int64']])
data_num = data_num.drop(columns = data_num.columns.tolist()[:data_num.columns.tolist().index('Ast/90')])


## Clasificación

In [11]:
data_clasif = data_proc.drop(columns = [col for col in data_proc.columns if data_proc[col].dtype not in ['float64','int64']])
data_clasif = data_clasif.drop(columns = data_num.columns.tolist()[:data_num.columns.tolist().index('Ast/90')]).join(data_proc['Pos'].apply(lambda x: x.split(',')[0])).rename(columns = {'Pos': 'Pos_filt'})

X_train, X_test, y_train, y_test = train_test_split(data_num, data_clasif['Pos_filt'], test_size=0.2, random_state=42)

clsf = GridSearchCV(estimator = KNeighborsClassifier(),param_grid={'n_neighbors':[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]}, cv = LeaveOneOut(), verbose=1)
clsf.fit(X_train,y_train)

Fitting 1567 folds for each of 19 candidates, totalling 29773 fits


GridSearchCV(cv=LeaveOneOut(), estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                         14, 15, 16, 17, 18, 19, 20]},
             verbose=1)

In [15]:
model = KNeighborsClassifier(n_neighbors = 8)
model.fit(X_train,y_train)
score = model.score(X_test, y_test)
print(score)


0.8341836734693877
